# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Mollière  ->  P. Mollière  |  ['P. Mollière']
M. Prunier  ->  M. Prunier  |  ['M. Prunier']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Stuber  ->  S. Stuber  |  ['S. Stuber']
J. Li  ->  J. Li  |  ['J. Li']


T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 92 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.21364


extracting tarball to tmp_2410.21364... done.
Retrieving document from  https://arxiv.org/e-print/2410.21366
extracting tarball to tmp_2410.21366...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/A1_Atlas_Of_cavities_v4.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[trim=9cm 8cm 0cm 10cm, clip, width=\paperwidth]{figures/A1_Atlas_Of_cavities_v4.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/A1_Hist_Fract_cavities.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{figures/A1_Hist_Fract_cavities.pdf}
  war

...

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure data-plxerr
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure data-plxerr as tmp_2410.22250/fig/data-plxerr.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-radial-sharp-nogse-1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-radial-sharp-nogse-1 as tmp_2410.22250/fig/model-radial-sharp-nogse-1.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedto

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-radial-flat-sc-1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-radial-flat-sc-1 as tmp_2410.22250/fig/model-radial-flat-sc-1.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-radial-flat-sc-2
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-radial-flat-sc-2 as tmp_2410.22250/fig/model-radial-flat-sc-2.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fn

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-flat-bpl-summary
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-flat-bpl-summary as tmp_2410.22250/fig/model-flat-bpl-summary.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-radial-flat-bpl-0
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-radial-flat-bpl-0 as tmp_2410.22250/fig/model-radial-flat-bpl-0.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as 

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-flat-dbpld-summary
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-flat-dbpld-summary as tmp_2410.22250/fig/model-flat-dbpld-summary.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure model-radial-flat-dbpld-0
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure model-radial-flat-dbpld-0 as tmp_2410.22250/fig/model-radial-flat-dbpld-0.pdf
  warnings.warn(LatexWarning(f'Recovered figure

Found 117 bibliographic references in tmp_2410.22250/paper.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21366-b31b1b.svg)](https://arxiv.org/abs/2410.21366) | **X-ray cavities in TNG-Cluster: AGN phenomena in the full cosmological context**  |
|| <mark>M. Prunier</mark>, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *Submitted to MNRAS. See further TNG-Cluster papers from Eric Rohr, and Urmila Chadayammuri over the next days, and on the TNG-Cluster website: this https URL*|
|**Abstract**|            Active galactic nuclei (AGN) feedback from supermassive black holes (SMBHs) at the centers of galaxy clusters plays a key role in regulating star formation and shaping the intracluster medium (ICM), often manifesting through prominent X-ray cavities embedded in the cluster's hot atmosphere. Here we show that X-ray cavities arise naturally due to AGN feedback in TNG-Cluster. This is a new suite of magnetohydrodynamic cosmological simulations of galaxy formation and evolution, and hence of galaxy clusters, whereby cold dark matter, baryon dynamics, galactic astrophysics, and magnetic fields are evolved together consistently. We construct mock Chandra X-ray observations of the central regions of the 352 simulated clusters at $z=0$ and find that $\sim$39 per cent contain X-ray cavities. Identified X-ray cavities vary in configuration (single, pairs, or multiples), with some still attached to SMBHs, while others have buoyantly risen. Their size ranges from a few to several tens of kpc. In terms of gas physical properties, TNG-Cluster X-ray cavities are underdense compared to the surrounding halo and filled with hot gas ($\sim$10$^8$K); 25 per cent of them are surrounded by an X-ray bright and compressed rim associated with a weak shock (Mach number $\sim 1.5$). Clusters exhibiting X-ray cavities are preferentially strong or weak cool-cores, are dynamically relaxed, and host SMBHs accreting at low Eddington rates. We show that TNG-Cluster X-ray cavities originate from episodic, wind-like energy injections from central AGN. Our results illustrate the existence and diversity of X-ray cavities simulated in state-of-the-art models within realistic cosmological environments and show that these can form without necessarily invoking bipolar, collimated, or relativistic jets.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21399-b31b1b.svg)](https://arxiv.org/abs/2410.21399) | **CO isotopologue-derived molecular gas conditions and CO-to-H$_2$ conversion factors in M51**  |
|| J. d. Brok, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *accepted for publication in AJ; 31 pages, 16 figures, 7 tables*|
|**Abstract**|            Over the past decade, several millimeter interferometer programs have mapped the nearby star-forming galaxy M51 at a spatial resolution of ${\le}170$ pc. This study combines observations from three major programs: the PdBI Arcsecond Whirlpool Survey (PAWS), the SMA M51 large program (SMA-PAWS), and the Surveying the Whirlpool at Arcseconds with NOEMA (SWAN). The dataset includes the (1-0) and (2-1) rotational transitions of $^{12}$CO, $^{13}$CO, and C$^{18}$O isotopologues. The observations cover the $r{<}\rm 3\,kpc$ region including center and part of the disk, thereby ensuring strong detections of the weaker $^{13}$CO and C$^{18}$O lines. All observations are convolved in this analysis to an angular resolution of 4$''$, corresponding to a physical scale of ${\sim}$170 pc. We investigate empirical line ratio relations and quantitatively evaluate molecular gas conditions such as temperature, density, and the CO-to-H$_2$ conversion factor ($\alpha_{\rm CO}$). We employ two approaches to study the molecular gas conditions: (i) assuming local thermal equilibrium (LTE) to analytically determine the CO column density and $\alpha_{\rm CO}$, and (ii) using non-LTE modeling with RADEX to fit physical conditions to observed CO isotopologue intensities. We find that the $\alpha_{\rm CO}$ values {in the center and along the inner spiral arm} are $\sim$0.5 dex (LTE) and ${\sim}$0.1 dex (non-LTE) below the Milky Way inner disk value. The average non-LTE $\alpha_{\rm CO}$ is $2.4{\pm}0.5$ M$_\odot$ pc$^{-2}$ (K km s$^{-1}$)$^{-1}$. While both methods show dispersion due to underlying assumptions, the scatter is larger for LTE-derived values. This study underscores the necessity for robust CO line modeling to accurately constrain the molecular ISM's physical and chemical conditions in nearby galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.22036-b31b1b.svg)](https://arxiv.org/abs/2410.22036) | **GaiaUnlimited: The old stellar disc of the Milky Way as traced by the Red Clump**  |
|| S. Khanna, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *27 pages, submitted to A&A*|
|**Abstract**|            We present an exploration of the Milky Way's structural parameters using an all-sky sample of RC giants to map the stellar density from the inner to the outer parts of the Galactic disc. These evolved giants are considered to be standard candles due to their low intrinsic variance in their absolute luminosities, allowing us to estimate their distances with reasonable confidence. We exploit all-sky photometry from the AllWISE mid-infrared survey and the Gaia survey, along with astrometry from Gaia Data Release 3 and recent 3D extinction maps, to develop a probabilistic scheme in order to select with high confidence \rc{}-like stars. Our curated catalogue contains about 10 million sources, for which we estimate photometric distances based on the WISE $W1$ photometry. We then derive the selection function for our sample, which is the combined selection function of sources with both \gaia{} and \allwise{} photometry. Using the distances and accounting for the full selection function of our observables, we are able to fit a two-disc, multi-parameter model to constrain the scale height (\hz{}), scale-length (\rd{}), flaring, and the relative mass ratios of the two disc components. We illustrate and verify our methodology using mock catalogues of \rc{} stars. We find that the \rc{} population is best described by a flared thin disc with scale length \rd{}=$3.56\pm0.32$ kpc and scale height at the Sun of \hzsun{}=$0.17\pm0.01$ kpc, and a shorter and thicker disc with \rd{}=$2.59\pm0.11$ kpc, \hzsun{}=$0.45\pm0.11$ kpc, with no flare. The thicker disc constitutes 64\% of the \rc{} stellar mass beyond 3 kpc, while the thin disk shows evidence of being warped beyond 9 kpc from the Galactic center. The residuals between the predicted number density of RC stars from our axisymmetric model and the measured counts show possible evidence of a two-armed spiral perturbation in the disc of the Milky Way.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.22250-b31b1b.svg)](https://arxiv.org/abs/2410.22250) | **The realm of Aurora. Density distribution of metal-poor giants in the heart of the Galaxy**  |
|| E. P. Kurbatov, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            The innermost portions of the Milky Way's stellar halo have avoided scrutiny until recently. The lack of wide-area survey data, made it difficult to reconstruct an uninterrupted view of the density distribution of the metal-poor stars inside the Solar radius. In this study, we utilize red giant branch (RGB) stars from Gaia, with metallicities estimated using spectro-photometry from Gaia Data Release 3. Accounting for Gaia's selection function, we examine the spatial distribution of metal-poor ([M/H]<-1.3) RGB stars, from the Galactic centre (r~1 kpc) out to beyond the Solar radius (r~18 kpc). Our best-fitting single-component cored power-law model shows a vertical flattening of ~0.5 and a slope -3.4, consistent with previous studies. Motivated by the mounting evidence for two distinct stellar populations in the inner halo, we additionally test a range of two-component models. One of the components models the tidal debris from the Gaia Sausage/Enceladus merger, while the other captures the Aurora population -- stars that predate the Galactic disk formation. Our best-fit two-component model suggests that both populations contribute equally around the Solar radius, but Aurora dominates the inner halo with a steeper power-law index of -4.5, in agreement with the nitrogen-rich star distribution measured by Horta et al. (2021).         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21364-b31b1b.svg)](https://arxiv.org/abs/2410.21364) | **easyCHEM: A Python package for calculating chemical equilibrium abundances in exoplanet atmospheres**  |
|| E. Lei, <mark>P. Mollière</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *Submitted for review to the Journal of Open Source Software (JOSS)*|
|**Abstract**|            For modeling the spectra of exoplanets one must know their atmospheric composition. This is necessary because the abundance of molecules, atoms, ions and condensates is needed to construct the total cross-section for the interaction between electro-magnetic radiation and matter. In addition, when solving for the temperature structure of an atmosphere the so-called adiabatic temperature gradient must be known, which describes the pressure-temperature dependence in convectively unstable regions well. Depending on the planetary properties, the composition and adiabatic gradients may be well described by equilibrium chemistry, which means that chemical reactions occur faster than any other relevant processes in the atmosphere, such as mixing. What is more, the equilibrium assumption often serves as a useful starting point for non-equilibrium calculations. Efficient and easy-to-use codes for determining equilibrium abundances are therefore needed. Here we report on our easyCHEM Python package that calculates atmospheric compositions and adiabatic temperature gradients in chemical equilibrium for any user-specified elemental composition.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21617-b31b1b.svg)](https://arxiv.org/abs/2410.21617) | **Einstein Probe discovery of EP240408a: a peculiar X-ray transient with an intermediate timescale**  |
|| W. Zhang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-30*|
|*Comments*| *25 pages, 11 figures*|
|**Abstract**|            We report the discovery of a peculiar X-ray transient, EP240408a, by Einstein Probe (EP) and follow-up studies made with EP, Swift, NICER, GROND, ATCA and other ground-based multi-wavelength telescopes. The new transient was first detected with Wide-field X-ray Telescope (WXT) on board EP on April 8th, 2024, manifested in an intense yet brief X-ray flare lasting for 12 seconds. The flare reached a peak flux of 3.9x10^(-9) erg/cm2/s in 0.5-4 keV, about 300 times brighter than the underlying X-ray emission detected throughout the observation. Rapid and more precise follow-up observations by EP/FXT, Swift and NICER confirmed the finding of this new transient. Its X-ray spectrum is non-thermal in 0.5-10 keV, with a power-law photon index varying within 1.8-2.5. The X-ray light curve shows a plateau lasting for about 4 days, followed by a steep decay till becoming undetectable about 10 days after the initial detection. Based on its temporal property and constraints from previous EP observations, an unusual timescale in the range of 7-23 days is found for EP240408a, which is intermediate between the commonly found fast and long-term transients. No counterparts have been found in optical and near-infrared, with the earliest observation at 17 hours after the initial X-ray detection, suggestive of intrinsically weak emission in these bands. We demonstrate that the remarkable properties of EP240408a are inconsistent with any of the transient types known so far, by comparison with, in particular, jetted tidal disruption events, gamma-ray bursts, X-ray binaries and fast blue optical transients. The nature of EP240408a thus remains an enigma. We suggest that EP240408a may represent a new type of transients with intermediate timescales of the order of about 10 days. The detection and follow-ups of more of such objects are essential for revealing their origin.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.21366/./figures/clusterMaps_default_tng_8192_99_ID-16921354_200_kpc_10.png', '', '', '', 'tmp_2410.21366/./figures/A1_Demog_tcool_Lx500.png', 'tmp_2410.21366/./figures/A1_Demog_XrayOff_Tcool.png', 'tmp_2410.21366/./figures/A1_subboxe_panel_2008.png', 'tmp_2410.21366/./figures/2008_subplots_UM.png', 'tmp_2410.21366/./figures/A1_gas_cells_KE_2008.png', 'tmp_2410.21366/./figures/A1_gas_cells_outflows_2008.png', '']
copying  tmp_2410.21366/./figures/clusterMaps_default_tng_8192_99_ID-16921354_200_kpc_10.png to _build/html/
file not found 
file not found 
file not found 
copying  tmp_2410.21366/./figures/A1_Demog_tcool_Lx500.png to _build/html/
copying  tmp_2410.21366/./figures/A1_Demog_XrayOff_Tcool.png to _build/html/
copying  tmp_2410.21366/./figures/A1_subboxe_panel_2008.png to _build/html/
copying  tmp_2410.21366/./figures/2008_subplots_UM.png to _build/html/
copying  tmp_2410.21366/./figures/A1_gas_cells_KE_2008.png to _build/html/
copying  tmp_2410.21366/./fi

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mvir}{M_{\rm 200c}}$
$\newcommand{\mvirhost}{\mvir^{\rm host}}$
$\newcommand{\rvir}{R_{\rm 200c}}$
$\newcommand{\rvirhost}{\rvir^{\rm host}}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\mgassat}{\mgas^{\rm sat}}$
$\newcommand{\mcgas}{M_{\rm ColdGas}}$
$\newcommand{\mcgassat}{\mcgas^{\rm sat}}$
$\newcommand{\mhgas}{M_{\rm HotGas}}$
$\newcommand{\mhgassat}{\mhgas^{\rm sat}}$
$\newcommand{\mcgassub}{\mcgas^{\rm subhalo}}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\mstarsat}{\mstar^{\rm sat}}$
$\newcommand{\mcgm}{M_{\rm CGM}}$
$\newcommand{\mcgmsat}{\mcgm^{\rm sat}}$
$\newcommand{\mism}{M_{\rm ISM}}$
$\newcommand{\mismsat}{\mism^{\rm sat}}$
$\newcommand{\rhalfstar}{R_{\rm half,\star}}$
$\newcommand{\rgal}{R_{\rm gal}}$
$\newcommand{\dsathost}{d_{\rm sat}^{\rm host}}$
$\newcommand{\sublink}{\textsc{sublink}}$
$\newcommand{\sublinkgal}{\sublink\_\textsc{gal} }$
$\newcommand{\subfind}{\textsc{subfind} }$
$\newcommand{\msun}{ M_{\odot}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# X-ray cavities in TNG-Cluster: AGN phenomena in the full cosmological context

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.21366-b31b1b.svg)](https://arxiv.org/abs/2410.21366)<mark>Appeared on: 2024-10-30</mark> -  _Submitted to MNRAS. See further TNG-Cluster papers from Eric Rohr, and Urmila Chadayammuri over the next days, and on the TNG-Cluster website: this https URL_

</div>
<div id="authors">

<mark>M. Prunier</mark>, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Active galactic nuclei (AGN) feedback from supermassive black holes (SMBHs) at the centers of galaxy clusters plays a key role in determining the properties of the intracluster medium (ICM) and in regulating star formation, often manifesting through prominent X-ray cavities embedded in the cluster's hot atmosphere. Here we show that X-ray cavities arise naturally due to AGN feedback in TNG-Cluster. This is a new suite of magnetohydrodynamic cosmological simulations of galaxy formation and evolution, and hence of galaxy clusters, whereby cold dark matter, baryon dynamics, galactic astrophysics, and magnetic fields are evolved together consistently. We construct mock Chandra X-ray observations of the central region of the 352 galaxy clusters in the simulation at $z=0$ and inspect them for X-ray cavities as typically done in observations. We show that X-ray cavities are common in TNG-Cluster, with $\sim$ 39 per cent of the simulated clusters featuring one or more X-ray cavities. Identified X-ray cavities come in a variety of configurations, including single, pairs, and multiples. Some are still attached to SMBHs, while others have buoyantly risen to larger distances. Their size ranges from a few to several tens of kpc. In terms of gas physical properties, TNG-Cluster X-ray cavities are underdense compared to the surrounding halo and filled with hot gas ( $\sim$ 10 $^8$ K); 25 per cent of them are surrounded by an X-ray bright and compressed rim associated with a weak shock (Mach number $\sim 1.5$ ). Clusters exhibiting X-ray cavities are preferentially strong or weak cool-cores, are dynamically relaxed, and host SMBHs accreting at low Eddington rates. We show that TNG-Cluster X-ray cavities originate from episodic, wind-like energy injections from central AGN. Our results illustrate the existence and diversity of X-ray cavities simulated in state-of-the-art models within realistic cosmological environments and show that these can form without necessarily invoking bipolar, collimated, or relativistic jets.

</div>

<div id="div_fig1">

<img src="tmp_2410.21366/./figures/clusterMaps_default_tng_8192_99_ID-16921354_200_kpc_10.png" alt="Fig4" width="100%"/>

**Figure 4. -** Maps of the thermodynamic and kinematic properties of the gas within an example TNG-Cluster system (Halo 16921354; maps side length: 200 kpc, depth: 20 kpc). Two X-ray cavities are visible: the first is attached to the central SMBH and exhibits a bright rim, while the second is detached and rising in the ICM. These X-ray cavities appear as under-dense and under-X-ray luminous regions filled with hot gas. Additionally, the attached X-ray cavity has an over-pressurized edge and a weak shock front with a Mach number of $\sim$ 1.3 (mass-weighted average Mach number along the line of sight, according to the shock finder: here the averages are obtained by only considering gas cells with shock Mach number larger than 0.9), while the detached X-ray cavity does not display such a feature. This panel illustrates the varying thermodynamic states of the gas within and surrounding the X-ray cavities. It highlights the typical properties of the X-ray cavity population identified in the TNG-Cluster simulation. (*fig:panel_cav_phys_pro*)

</div>
<div id="div_fig2">

<img src="" alt="Fig5.1" width="20%"/><img src="" alt="Fig5.2" width="20%"/><img src="" alt="Fig5.3" width="20%"/><img src="tmp_2410.21366/./figures/A1_Demog_tcool_Lx500.png" alt="Fig5.4" width="20%"/><img src="tmp_2410.21366/./figures/A1_Demog_XrayOff_Tcool.png" alt="Fig5.5" width="20%"/>

**Figure 5. -** Demographics of clusters with and without identified X-ray cavities in the TNG-Cluster simulation at $z=0$. _Top left:_ Percentage of clusters with identified X-ray cavities. _Top right:_ Among clusters with identified X-ray cavities, percentage of clusters having one, two, three, or four X-ray cavities.     _Bottom Left:_ Percentage distribution of cool-core (CC), weak cool-core (WCC), and non-cool-core (NCC) clusters for each category of clusters with or without identified X-ray cavities. The plot shows, for clusters with two X-ray cavities (x-label "2"), that 54 per cent of the TNG-Cluster halos hosting two X-ray cavities are SCCs and 46 per cent WCCs. _ Bottom middle:_ Central cooling time vs. X-ray luminosity within $r_{500\text{c}}$ for each halo, color-coded by the number of identified X-ray cavities. _Bottom Right:_ Central cooling time vs. the distance between the SMBH and the X-ray luminosity peak, i.e. a measure of un/relaxedness, color-coded by the number of identified X-ray cavities. In TNG-Cluster, $\sim$ 39 per cent of clusters exhibit at least one X-ray cavity. Clusters with identified X-ray cavities tend to have shorter central cooling times and higher X-ray luminosities, and to be more relaxed (smaller X-ray peak offset). (*fig:panel*)

</div>
<div id="div_fig3">

<img src="tmp_2410.21366/./figures/A1_subboxe_panel_2008.png" alt="Fig6.1" width="20%"/><img src="tmp_2410.21366/./figures/2008_subplots_UM.png" alt="Fig6.2" width="20%"/><img src="tmp_2410.21366/./figures/A1_gas_cells_KE_2008.png" alt="Fig6.3" width="20%"/><img src="tmp_2410.21366/./figures/A1_gas_cells_outflows_2008.png" alt="Fig6.4" width="20%"/><img src="" alt="Fig6.5" width="20%"/>

**Figure 6. -** Time evolution of selected quantities from the TNG300 most massive cluster, for which high temporal cadence output is available. The top two rows show six X-ray emission maps of the central region of the cluster, across 55 Myr, featuring two X-ray cavity events occurring in the second and last frame: mock Chandra surface brightness maps and the same maps processed with an unsharp mask filter to highlight X-ray-depleted regions, respectively. From top to bottom, the lower three panels quantify the time evolution of: _i)_ energy injected in kinetic mode by the SMBH between two successive snapshots, i.e. across time spans of about 10 Myrs; _ii)_ maximum radial outflow velocities (95th percentile) of the gas at different distances from the SMBH; and _iii)_ SMBH accretion rate. In all panels, the two dotted vertical lines indicate the frames just before the first and second X-ray cavities appear. The two events are characterized by a 10$^{42-43}$ erg s$^{-1}$ kinetic energy release and high-velocity gas outflows, as well as concomitant decreases in the SMBH accretion rate. See continuation in Figure \ref{fig:subboxe_1gyr}. (*fig:subboxe_panel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.21366"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# CO isotopologue-derived molecular gas conditions and CO-to-$H_2$ conversion factors in M51

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.21399-b31b1b.svg)](https://arxiv.org/abs/2410.21399)<mark>Appeared on: 2024-10-30</mark> -  _accepted for publication in AJ; 31 pages, 16 figures, 7 tables_

</div>
<div id="authors">

J. d. Brok, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark>

</div>
<div id="abstract">

**Abstract:** Over the past decade, several millimeter interferometer programs have mapped the nearby star-forming galaxy M51 at a spatial resolution of ${\le}170$ pc. This study combines observations from three major programs: the $*PdBI Arcsecond Whirlpool Survey*$ (PAWS), the SMA M51 large program (SMA-PAWS), and the $*Surveying the Whirlpool at Arcseconds with NOEMA*$ (SWAN). The dataset includes the (1-0) and (2-1) rotational transitions of $^{12}$ CO, $^{13}$ CO, and C $^{18}$ O isotopologues. ${The observations cover the $r{<}\rm 3 kpc$ region including center and part of the disk, thereby ensuring strong detections of the weaker $^{13}$CO and C$^{18}$O lines.}$ All observations are convolved in this analysis to an angular resolution of 4 $\arcsec$ , corresponding to a physical scale of ${\sim}$ 170 pc. We investigate empirical line ratio relations and quantitatively evaluate molecular gas conditions such as temperature, density, and the CO-to-$H_2$ conversion factor ( $\alpha_{\rm CO}$ ). We employ two approaches to study the molecular gas conditions: (i) assuming local thermal equilibrium (LTE) to analytically determine the CO column density and $\alpha_{\rm CO}$ , and (ii) using non-LTE modeling with \texttt{RADEX} to fit physical conditions to observed CO isotopologue intensities. We find that the $\alpha_{\rm CO}$ values ${in the center and along the inner spiral arm}$ are $\sim$ 0.5 dex (LTE) and ${\sim}$ 0.1 dex (non-LTE) below the Milky Way inner disk value. The average non-LTE $\alpha_{\rm CO}$ is $2.4{\pm}0.5$ M $_\odot$ pc $^{-2}$ (K km s $^{-1}$ ) $^{-1}$ . While both methods show dispersion due to underlying assumptions, the scatter is larger for LTE-derived values. This study underscores the necessity for robust CO line modeling to accurately constrain the molecular ISM's physical and chemical conditions in nearby galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2410.21399/./Figures/ratio_1.png" alt="Fig8" width="100%"/>

**Figure 8. -** ** CO Isotopologue Ratio Trends** These panels show the line ratio as a function of galactocentric radius (left), SFR surface density (middle), and FWHM of $^{12}$CO(1-0) (right) panel. Only lines of sight within the NOEMA field-of-view are considered. We show the individual sightlines where both lines have $\rm S/N{\ge}5$ as colored points. The small black triangles indicate 5$\sigma$ upper limits. In addition, the orange line depicts the stacked line ratio trend. We stack all points for which $^{12}$CO(1-0) is significantly detected. The red line in the left panels is the radial trend obtained at low-angular resolution from \citetalias{denBrok2022}. The red shaded region in the right panels indicates where the FWHM is below two times the channel width of our data (10 km s$^{-1}$). The histograms show the distribution of these significantly detected data points per line ratio. The black line represents the weighted average (weighted by the $^{12}$CO(1-0) intensity) and the dotted lines the weighted 16$^{\rm th}$-to-84$^{\rm th}$ percentile range. (*fig:rgal_ratio*)

</div>
<div id="div_fig2">

<img src="tmp_2410.21399/./Figures/outine_M51.png" alt="Fig1" width="100%"/>

**Figure 1. -** ** Spatial Coverage of Observations** In this study, we rely on Plateau de Bure Interferometer (PdBI), Northern Extended Millimeter Array (NOEMA), and Submillimeter Array (SMA) observations, each of which covered some extent of the disk of M51. The boxes indicate which lines were observed by each respective program. Background HST image credit: NASA, ESA, S. Beckwith (STScI), and The Hubble Heritage Team (STScI/AURA). (*fig:outline*)

</div>
<div id="div_fig3">

<img src="tmp_2410.21399/./Figures/ratio_2.png" alt="Fig9" width="100%"/>

**Figure 9. -** ** CO Isotopologue Ratio Trends (continued).** These panels have the same description as in \autoref{fig:rgal_ratio}. (*fig:rgal_ratio_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.21399"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\ensuremath{\rm  km s^{-1}}\xspace}$
$\newcommand{\masyr}{\ensuremath{\rm  mas yr^{-1}}\xspace}$
$\newcommand{\kmskpc}{\ensuremath{\rm  km s^{-1} kpc^{-1}\xspace}}$
$\newcommand{\teff}{\ensuremath{{T_{\rm eff}}}\xspace}$
$\newcommand{\lum}{\ensuremath{{L}}\xspace}$
$\newcommand{\loglum}{\ensuremath{\log\lum}\xspace}$
$\newcommand{\loggrav}{\ensuremath{\log g}\xspace}$
$\newcommand{\mass}{\ensuremath{{M}}\xspace}$
$\newcommand{\radius}{\ensuremath{{R}}\xspace}$
$\newcommand{\Zabun}{\ensuremath{{Z}}\xspace}$
$\newcommand{\azero}{\ensuremath{A_0}\xspace}$
$\newcommand{\rzero}{\ensuremath{R_0}\xspace}$
$\newcommand{\ebpminrp}{\ensuremath{E(G_{\rm BP} - G_{\rm RP})}\xspace}$
$\newcommand{\gmag}{\ensuremath{G}}$
$\newcommand{\bpmag}{\ensuremath{G_\mathrm{BP}}}$
$\newcommand{\rpmag}{\ensuremath{G_\mathrm{RP}}}$
$\newcommand{\mg}{M_\gmag}$
$\newcommand{\ag}{\ensuremath{A_G}\xspace}$
$\newcommand{\ra}{\ensuremath{\alpha}}$
$\newcommand{\dec}{\ensuremath{\delta}}$
$\newcommand{\parallax}{\ensuremath{\varpi}}$
$\newcommand{\sigparallax}{\ensuremath{\sigma_{\varpi}}}$
$\newcommand{\parzp}{\ensuremath{\varpi_{\rm zp}}}$
$\newcommand{\pmra}{\ensuremath{\mu_{\ra\ast}}}$
$\newcommand{\pmdec}{\ensuremath{\mu_\dec}}$
$\newcommand{\propm}{\ensuremath{\mu}}$
$\newcommand{\glon}{\ensuremath{\ell}}$
$\newcommand{\glat}{\ensuremath{b}}$
$\newcommand{\mag}{\ensuremath{ \mathrm{mag}}\xspace}$
$\newcommand{\nm}{\ensuremath{ \mathrm{nm}}\xspace}$
$\newcommand{\um}{\ensuremath{ \mu\mathrm{m}}\xspace}$
$\newcommand{\erg}{\ensuremath{ \rm erg}\xspace}$
$\newcommand{\ergs}{\ensuremath{ \rm erg~s}\xspace}$
$\newcommand{\yr}{\ensuremath{ \rm yr}\xspace}$
$\newcommand{\Myr}{\ensuremath{ \rm Myr}\xspace}$
$\newcommand{\Gyr}{\ensuremath{ \rm Gyr}\xspace}$
$\newcommand{\pc}{\ensuremath{ \rm pc}\xspace}$
$\newcommand{\kpc}{\ensuremath{ \rm kpc}\xspace}$
$\newcommand{\Mpc}{\ensuremath{ \rm Mpc}\xspace}$
$\newcommand{\Lsun}{\ensuremath{ {\lum}_{\odot}}\xspace}$
$\newcommand{\Msun}{\ensuremath{ {\mass}_{\odot}}\xspace}$
$\newcommand{\Rsun}{\ensuremath{ {\radius}_{\odot}}\xspace}$
$\newcommand{\Zsun}{\ensuremath{ {\Zabun}_{\odot}}\xspace}$
$\newcommand{\ang}{\ensuremath{ \text{Å}}\xspace}$
$\newcommand{\mas}{\ensuremath{ \mathrm{mas}}\xspace}$
$\newcommand{\kms}{\ensuremath{\textrm{km s}^{-1}}}$
$\newcommand{\maspyr}{\ensuremath{\textrm{mas yr}^{-1}}}$
$\newcommand{\masyr}{\ensuremath{\textrm{mas yr}^{-1}}}$
$\newcommand{\msun}{\mass_\odot}$
$\newcommand{◦ee}{\ensuremath{^\circ}}$
$\newcommand{\rpmag}{\ensuremath{G_\mathrm{RP}}}$
$\newcommand$
$\newcommand$
$\newcommand{\healpix}{{\sl HEALPix}}$
$\newcommand{\galaxia}{{\sl Galaxia}}$
$\newcommand{\agama}{{\sl AGAMA}}$
$\newcommand{\parsec}{{\sl PARSEC}}$
$\newcommand{\esagaia}{{\sl Gaia}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\absg}{M_{G}}$
$\newcommand{\absw}{M_{W1}}$
$\newcommand{\gunlim}{\textit{GaiaUnlimited}}$
$\newcommand{\absmag}{M_{\lambda}}$
$\newcommand{\appmag}{m_{\lambda}}$
$\newcommand{\appmaglim}{m_{\lambda, lim}}$
$\newcommand{\alambda}{A_{\lambda}}$
$\newcommand{\twomass}{\textit{2MASS}}$
$\newcommand{\panstars}{\textit{Pan-STARRS}}$
$\newcommand{\allwise}{\textit{AllWISE}}$
$\newcommand{\wise}{\textit{WISE}}$
$\newcommand{\bayestar}{\textit{Bayestar}}$
$\newcommand{\gaussian}{\textit{Gaussian}}$
$\newcommand{\hi}{\mbox{H{\sc i}}}$
$\newcommand{\los}{l.o.s}$
$\newcommand{\alfe}{\ensuremath{[\mathrm{\alpha/Fe}]}}$
$\newcommand{\feh}{\ensuremath{[\mathrm{Fe/H}]}}$
$\newcommand{\dex}{\ensuremath{ \mathrm{dex}}}$
$\newcommand{\logg}{\mbox{\log g}}$
$\newcommand{\kiel}{{\sl Kiel}}$
$\newcommand{\corehel}{{\sc CHeB}}$
$\newcommand{\jkzero}{(J-K)_{0}}$
$\newcommand{\clr}{(J-K)_{0}}$
$\newcommand{\camd}{CaMD}$
$\newcommand{\cmd}{CMD}$
$\newcommand{\hpix}{HEALPix}$
$\newcommand{\AC}{{AC20-\piepi>3 }}$
$\newcommand{\ACf}{{AC20-\piepi>3}}$
$\newcommand{\bprp}{{ G_\mathrm{BP} - G_\mathrm{RP}}}$
$\newcommand{\grp}{{ G - G_\mathrm{RP}}}$
$\newcommand{\gaiaG}{{G}}$
$\newcommand{\ruwe}{\textit{RUWE}}$
$\newcommand{\astromexnoise}{\textit{astrometric\_excess\_noise }}$
$\newcommand{\visibperiods}{\textit{visibility\_periods\_used }}$
$\newcommand{\errorovparallax}{\sigma_{\varpi}/\varpi}$
$\newcommand{\cbj}{{\sl CBJ21}}$
$\newcommand{\schlegel}{{\sl S98}}$
$\newcommand{\geo}{{d_{\rm geo}}}$
$\newcommand{\photogeo}{{d_{\rm photgeo}}}$
$\newcommand{\mumaxdust}{\mu\_max_{i}}$
$\newcommand{\fsel}{{F}}$
$\newcommand{\fseli}{{F_{\rm i}}}$
$\newcommand{\fselitop}{{S_{\rm top, i}}}$
$\newcommand{\fselisub}{{S_{\rm sub, i}}}$
$\newcommand{\rcut}{{R_{\rm cut}}}$
$\newcommand{\rdtwo}{{R_{\rm d2}}}$
$\newcommand{\rd}{{R_{\rm d}}}$
$\newcommand{\hz}{{\sl h_{z}}}$
$\newcommand{\hztwo}{{\sl h_{z2,\odot}}}$
$\newcommand{\hzsun}{{\sl h_{z,\odot}}}$
$\newcommand{\logrflare}{{\log R_{fl}}}$
$\newcommand{\rflare}{\ensuremath{R_{\rm fl}}}$
$\newcommand{\fdisc}{{\sl f_{d1}}}$
$\newcommand{\zwarp}{{z_{\rm w}}}$
$\newcommand{\awarp}{{a_{\rm w}}}$
$\newcommand{\rwarp}{{R_{\rm w}}}$
$\newcommand{\hwarp}{{h_{\rm w0}}}$
$\newcommand{\phiwarp}{{\phi_{\rm w}}}$
$\newcommand{\phiprime}{\phi^{'}}$
$\newcommand{\rgal}{{\sl R}}$
$\newcommand{\zgal}{{\sl Z_{GC}}}$
$\newcommand{\xgc}{{\sl X_{GC}}}$
$\newcommand{\ygc}{{\sl Y_{GC}}}$
$\newcommand{\zgc}{{\sl Z_{GC}}}$
$\newcommand{\mir}{{MIR}}$
$\newcommand{\rdratio}{{\sl R_{d2}/R_{d1}}}$
$\newcommand{\hzratio}{{\sl h_{z2}/h_{z1}}}$
$\newcommand{\nrproj}{{\sl N(R|z=0)}}$
$\newcommand{\nzproj}{{\sl N(z|R)}}$
$\newcommand{\nmin}{{N_{\rm min, i}}}$
$\newcommand{\lucey}{{L20}}$
$\newcommand{\jie}{{Yu18}}$
$\newcommand{\elsworth}{{Els19}}$
$\newcommand{\andrae}{{A23}}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand$</div>



<div id="title">

# GaiaUnlimited: The old stellar disc of the Milky Way \\as traced by the Red Clump

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.22036-b31b1b.svg)](https://arxiv.org/abs/2410.22036)<mark>Appeared on: 2024-10-30</mark> -  _27 pages, submitted to A&A_

</div>
<div id="authors">

S. Khanna, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We present an exploration of the Milky Way's structural parameters using an all-sky sample of $\rcfull$ (RC) giants to map the stellar density from the inner to the outer parts of the Galactic disc. These evolved giants are considered to be standard candles due to their low intrinsic variance in their absolute luminosities, allowing us to estimate their distances with reasonable confidence. We exploit all-sky photometry from the AllWISE mid-infrared survey and the Gaia survey, along with astrometry from Gaia Data Release 3 and recent 3D extinction maps, to develop a probabilistic scheme in order to select with high confidence $\rc$ -like stars.Our curated catalogue contains about 10 million sources, for which we estimate photometric distances based on the WISE $W1$ photometry. We then derive the selection function for our sample, which is the combined selection function of sources with both $\gaia$ and $\allwise$ photometry.  Using the distances and accounting for the full selection function of our observables, we are able to fit a two-disc, multi-parameter model to constrain the scale height ( $\hz$ ), scale-length ( $\rd$ ), flaring, and the relative mass ratios of the two disc components. We illustrate and verify our methodology using mock catalogues of $\rc$ stars. We find that the $\rc$ population is best described by a flared thin disc with scale length $\rd$ = $3.56\pm0.32$ kpc and scale height at the Sun of $\hzsun$ = $0.17\pm0.01$ kpc, and a shorter and thicker disc with $\rd$ = $2.59\pm0.11$ kpc, $\hzsun$ = $0.45\pm0.11$ kpc, with no flare. The thicker disc constitutes 64 \% of the $\rc$ stellar mass beyond 3 kpc, while the thin disk shows evidence of being warped beyond 9 kpc from the Galactic center. The residuals between the predicted number density of RC stars from our axisymmetric model and the measured counts show possible evidence of a two-armed spiral perturbation in the disc of the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2410.22036/./figures/sf_illus_nodust.png" alt="Fig10" width="100%"/>

**Figure 10. -** Illustration of the selection function projected in $\phi-$\rgal  space, for the case without dust at $\zgal$ =0 kpc (midplane). Panel (a) shows the map of the selection fraction computed using the sub-binning method, panel (b) shows the same for the method without sub-binning, while panel (c) shows the distance to all voxels that are within a distance of $d_{max} < 12.94 $\kpc$$ of the Sun. Panel (a) also shows the additional voxels that would be missed by assuming the median values of observables ($l,b,G$). In all panels, the location of the Sun is indicated by the black star. (*fig:sf_illus_nodust*)

</div>
<div id="div_fig2">

<img src="tmp_2410.22036/./figures/warp_visual_outerbt1.png" alt="Fig19" width="100%"/>

**Figure 19. -** Residuals between the model and data in the $\phi$,$\zgal$  projection, same as \autoref{fig:residuals_phiz_inner} but for the outer disc ($9.5$ kpc$<$\rgal$<14.5$ kpc). (*fig:residuals_phiz_outer*)

</div>
<div id="div_fig3">

<img src="tmp_2410.22036/./figures/residuals_polar_2.png" alt="Fig20.1" width="33%"/><img src="tmp_2410.22036/./figures/residuals_polar_0.png" alt="Fig20.2" width="33%"/><img src="tmp_2410.22036/./figures/residuals_polar_1.png" alt="Fig20.3" width="33%"/>

**Figure 20. -** Relative residual (1-model/data) for Model 2 from \autoref{tab:bestfitdatamodel}, applied to \gaiawise{[RC]}, and shown in polar coordinates. The residuals are shown for three slices in $\zgal$ , above the plane (panel a), in the midplane (panel b), and below the plane (panel c). The 2-arm NIR spiral model from \cite{Drimmel:2000} is overplotted as black curves. (*fig:residuals_polar*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.22036"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\note}[1]{\textit{\textcolor{red}{#1}}}$
$\newcommand{\mathdash}{ \text{---} }$
$\newcommand{\diff}[2]{{\frac{d{#1}}{d{#2}}}}$
$\newcommand{\pdiff}[2]{{\frac{\partial{#1}}{\partial{#2}}}}$
$\newcommand{\vec}[1]{{\bm{\mathrm{#1}}}}$</div>



<div id="title">

# The realm of Aurora. Density distribution of metal-poor giants in the heart of the Galaxy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.22250-b31b1b.svg)](https://arxiv.org/abs/2410.22250)<mark>Appeared on: 2024-10-30</mark> -  _Submitted to MNRAS_

</div>
<div id="authors">

E. P. Kurbatov, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** The innermost portions of the Milky Way's stellar halo have avoided scrutiny until recently. The lack of wide-area survey data, made it difficult to reconstruct an uninterrupted view of the density distribution of the metal-poor stars inside the Solar radius. In this study, we utilize red giant branch (RGB) stars from _Gaia_ , with metallicities estimated using spectrophotometry from _Gaia_ Data Release 3. Accounting for _Gaia_ 's selection function, we examine the spatial distribution of metal-poor ( [ M/H ] $<-1.3$ ) RGB stars, from the Galactic center ( $r \approx 1$ kpc) out to beyond the Solar radius ( $r \approx 18$ kpc). Our best-fitting single-component cored power-law model shows a vertical flattening of $\approx 0.5$ and a slope $\approx -3.4$ , consistent with previous studies. Motivated by the mounting evidence for two distinct stellar populations in the inner halo, we additionally test a range of two-component models. One of the components models the tidal debris from the _Gaia_ Sausage/Enceladus merger, while the other captures the Aurora population -- stars that predate the Galactic disk formation. Our best-fit two-component model suggests that both populations contribute equally around the Solar radius, but Aurora dominates the inner halo with a steeper power-law index of $\approx -4.5$ , in agreement with the nitrogen-rich star distribution measured by [ and Horta (2021)](https://ui.adsabs.harvard.edu/abs/2021MNRAS.500.5462H) .

</div>

<div id="div_fig1">

<img src="tmp_2410.22250/fig/model-radial-flat-sc-0.png" alt="Fig7" width="100%"/>

**Figure 7. -** DPL Aurora \& SC GS/E model. Radial density profiles for sample (blue line), for Aurora model (dashed orange line), for GS/E model (dashed grey), for sum of the last two (dashed black), and for total density after selections applied (solid black line). Bottom panel shows the relative residuals between the sample and the model after the selections applied: $(\mathtt{model} - \mathtt{data}) / (\mathtt{model} + \mathtt{data})$. (*fig:model-two-comp-radial*)

</div>
<div id="div_fig2">

<img src="tmp_2410.22250/fig/model-flat-sc-summary.png" alt="Fig6" width="100%"/>

**Figure 6. -** A two-component model with DPL Aurora \& SC GS/E. Similar to Figure \ref{fig:model-summary-nogse}, this gives  a summary of the best-fit model parameters in various metallicity bins, see \autoref{tab:summary-dpl}. ** Top left panel:** horizontal (blue) and vertical (red) scales of the model. ** Top right:** Flattening $a/b$. ** Bottom left:** Power-law index of the inner slope.  ** Bottom right:** Estimated stellar mass of the Aurora (orange) and the GS/E (grey) populations. (*fig:model-two-comp-params*)

</div>
<div id="div_fig3">

<img src="tmp_2410.22250/fig/data-kinem-all.png" alt="Fig14" width="100%"/>

**Figure 14. -** Orbital properties of the vetted RGB sample in [Andrae, Rix and Chandra (2023)](https://ui.adsabs.harvard.edu/abs/2023ApJS..267....8A). ** Left:** Column-normalized distribution of orbital circularity vs. metallicity. ** Middle:** orbital circularity vs. apocentric distance. ** Right:** energy vs. the vertical component of the angular momentum. In the last two panels, stars are subjected to a metallicity selection $-2.0 < [\mathrm{M}/\mathrm{H}] \leq -1.5$. The approximate locations of two halo populations are marked in colour: the Aurora stars are in blue, and the GS/E stars are in green. The stars likely captured by bar resonances  are shown in magenta. See Section \ref{sec:sample-selection} for further details. The dotted black line in the right panel shows the boundary  separating the in-situ and accreted populations according to [Belokurov and Kravtsov (2023)](https://ui.adsabs.harvard.edu/abs/2023MNRAS.525.4456B). (*fig:data-kinem-all*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.22250"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

235  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
